#### This notebook is a part of [solution](https://www.kaggle.com/competitions/image-matching-challenge-2022/discussion/328805) presented by team of 6th place.

Image size 1024 
* w/o TTA Public/Private 0.812/0.817
* with TTA (rot 15°) Public/Private 0.815/0.818

#### Please, upvote if you find it useful!

In [ ]:
import time
import numpy as np
import pandas as pd
import sys, os, csv
from PIL import Image
import cv2
import gc
import matplotlib.pyplot as plt
import torch
from IPython import display
import PIL
import sys
from pathlib import Path
import matplotlib.cm as cm
import torchvision

In [ ]:
src = '/kaggle/input/image-matching-challenge-2022/'

In [ ]:
dry_run = False

## Quadree attention

In [ ]:
%%capture
!cp -r /kaggle/input/quadtreeattention/ /kaggle/working/ # input folder is read only
!cd /kaggle/working/quadtreeattention/QuadTreeAttention-master/QuadTreeAttention/ && pip install .

sys.path.append('/kaggle/working/quadtreeattention/QuadTreeAttention-master/')
sys.path.append('/kaggle/working/quadtreeattention/QuadTreeAttention-master/FeatureMatching/')
sys.path.append('/kaggle/working/quadtreeattention/QuadTreeAttention-master/QuadTreeAttention/')

In [ ]:
%%capture
!pip install /kaggle/input/igla-py-wheels/loguru-0.5.3-py3-none-any.whl
!pip install /kaggle/input/igla-py-wheels/einops-0.4.1-py3-none-any.whl
!pip install /kaggle/input/igla-py-wheels/timm-0.4.12-py3-none-any.whl

## Init QuadTreeAttention

In [ ]:
from FeatureMatching.src.config.default import get_cfg_defaults
config = get_cfg_defaults()

# INDOOT lofrt_ds_quadtree config
config.LOFTR.MATCH_COARSE.MATCH_TYPE = 'dual_softmax'
config.LOFTR.MATCH_COARSE.SPARSE_SPVS = False
config.LOFTR.RESNETFPN.INITIAL_DIM = 128
config.LOFTR.RESNETFPN.BLOCK_DIMS=[128, 196, 256]
config.LOFTR.COARSE.D_MODEL = 256
config.LOFTR.COARSE.BLOCK_TYPE = 'quadtree'
config.LOFTR.COARSE.ATTN_TYPE = 'B'
config.LOFTR.COARSE.TOPKS=[32, 16, 16]
config.LOFTR.FINE.D_MODEL = 128
config.TRAINER.WORLD_SIZE = 1 # 8
config.TRAINER.CANONICAL_BS = 32
config.TRAINER.TRUE_BATCH_SIZE = 1
_scaling = 1
config.TRAINER.ENABLE_PLOTTING = False
config.TRAINER.SCALING = _scaling
config.TRAINER.TRUE_LR = 1e-3 # 1e-4 config.TRAINER.CANONICAL_LR * _scaling
config.TRAINER.WARMUP_STEP = 0 #math.floor(config.TRAINER.WARMUP_STEP / _scaling)

In [ ]:
%%capture
from FeatureMatching.src.utils.profiler import build_profiler
from FeatureMatching.src.lightning.lightning_loftr import PL_LoFTR


# lightning module
qta_center_img_mask = False
qta_max_img_size = 1056
qta_torch_device = torch.device('cpu' if not torch.cuda.is_available() else 'cuda')

qta_device = "cuda" if torch.cuda.is_available() else "cpu"
disable_ckpt = True
qta_profiler_name = None # help='options: [inference, pytorch], or leave it unset
qta_profiler = build_profiler(qta_profiler_name)
qta_model = PL_LoFTR(config,
                 pretrained_ckpt= "../input/quadtreecheckpoints/outdoor_quadtree.ckpt", # args.ckpt_path, from scratch atm
                 profiler=qta_profiler)
qta_matcher = qta_model.matcher
qta_matcher.eval()
qta_matcher.to(qta_device)

## Utils

In [ ]:
def load_loftr_image_origNEW(fname, img_w=480, img_h=832):
    img0_raw = cv2.imread(fname, cv2.IMREAD_GRAYSCALE)
    if img_w > 0 and img_h > 0:
        img0_raw = cv2.resize(img0_raw, (img_w, img_h))
    img0 = torch.from_numpy(img0_raw)[None][None].cuda() / 255.
    return img0


def load_resized_image(fname, max_image_size):
    img = cv2.imread(fname)
    if max_image_size == -1:
        # no resize
        return img, 1.0
    scale = max_image_size / max(img.shape[0], img.shape[1]) 
    w = int(img.shape[1] * scale)
    h = int(img.shape[0] * scale)
    img = cv2.resize(img, (w, h))
    return img, scale


def scale_to_resized(mkpts0, mkpts1, scale1, scale2):
    ### scale to original im size because we used max_image_size
    # first point
    mkpts0[:, 0] = mkpts0[:, 0] / scale1
    mkpts0[:, 1] = mkpts0[:, 1] / scale1
    
    # second point
    mkpts1[:, 0] = mkpts1[:, 0] / scale2
    mkpts1[:, 1] = mkpts1[:, 1] / scale2
    
    return mkpts0, mkpts1


def put_img_on_disk(img, output_img_tag):
    img_path_on_disk = f'/kaggle/working/{output_img_tag}.png'
    cv2.imwrite(img_path_on_disk, img)
    return img_path_on_disk

In [ ]:
def calc_divide_size_smallest(im_size, coef):
    # select size dividable by coef
    if im_size % coef == 0:
        # already dividable, just return original
        return im_size
    return round(((im_size / coef) + 0.5)) * coef


def add_zero_padding_two_img_same(img1, img2, div_coef=32):
    img1_height, img1_width, img1_channels = img1.shape
    img2_height, img2_width, img2_channels = img2.shape
    
    # fit both images on canvas
    max_width = max(img1_width, img2_width)
    max_height = max(img1_height, img2_height) 
    
    # use own width and height for image with zero-padding
    result1, offset1 = create_zero_padding_img(img1, max_width, max_height, img1_channels, div_coef)
    result2, offset2 = create_zero_padding_img(img2, max_width, max_height, img2_channels, div_coef)
    
    return result1, result2, offset1, offset2


def create_zero_padding_img(img, max_im_width, max_im_height, channels, div_coef=32):
    
    # create new image of desired size and color (black) for padding
    new_area_image_width = calc_divide_size_smallest(max_im_width, div_coef)
    new_area_image_height = calc_divide_size_smallest(max_im_height, div_coef)
    
    im_height, im_width, im_channels = img.shape
    if qta_center_img_mask:
        x_offset = (new_area_image_width - im_width) // 2
        y_offset = (new_area_image_height - im_height) // 2
    else:
        x_offset = 0
        y_offset = 0
    
    im_right = x_offset + im_width # right of image corner where ends
    im_bottom = y_offset + im_height # right of image corner where ends
    
    color = (0,0,0)
    result = np.full((new_area_image_height, new_area_image_width, im_channels), color, dtype=np.uint8)
    
    # copy img image into center of result image
    result[y_offset:im_bottom, x_offset:im_right] = img
    
    # return image and x,y of old image in a new image (frame)
    return result, (x_offset, y_offset)


def unpad_matches(mkpts0, mkpts1, offset_point1, offset_point2):
    offset_x1, offset_y1 = offset_point1
    offset_x2, offset_y2 = offset_point2

    # remove offeset
    mkpts0[:, 0] = mkpts0[:, 0] - offset_x1
    mkpts0[:, 1] = mkpts0[:, 1] - offset_y1
    
    mkpts1[:, 0] = mkpts1[:, 0] - offset_x2
    mkpts1[:, 1] = mkpts1[:, 1] - offset_y2
    return mkpts0, mkpts1

In [ ]:
def inf_qta(image_fpath_1, image_fpath_2, max_image_size=qta_max_img_size, divide_coef=32):

    # resize image if we need
    img1_resized, scale1 = load_resized_image(image_fpath_1, max_image_size)
    img2_resized, scale2 = load_resized_image(image_fpath_2, max_image_size)

    
    ### add padding -> use same image padding mask because models wants it
    pad_img1, pad_img2, pad_offset_p1, pad_offset_p2 = add_zero_padding_two_img_same(img1_resized, img2_resized, divide_coef)
    
        
    # save temporarily    
    img1_disk_path = put_img_on_disk(pad_img1, 'qta_img1')
    img2_disk_path = put_img_on_disk(pad_img2, 'qta_img2')
    
    
    # load withr loftr    
    gray_img_1 = load_loftr_image_origNEW(img1_disk_path, -1, -1)
    gray_img_2 = load_loftr_image_origNEW(img2_disk_path, -1, -1)
    
    batch = {'image0': gray_img_1, 'image1': gray_img_2}
    
    # Inference
    with torch.no_grad():
        qta_matcher.eval()
        qta_matcher.to(qta_device)
        
        qta_matcher(batch)
        mkpts0 = batch['mkpts0_f'].cpu().numpy()
        mkpts1 = batch['mkpts1_f'].cpu().numpy()
        mconf = batch['mconf'].cpu().numpy()
    
    ### unpad matches
    mkpts0, mkpts1 = unpad_matches(mkpts0, mkpts1, pad_offset_p1, pad_offset_p2)
    
    ### scale to original im size because we used max_image_size
    mkpts0, mkpts1 = scale_to_resized(mkpts0, mkpts1, scale1, scale2)
    
    # cleanup
    if os.path.exists(img1_disk_path): os.remove(img1_disk_path)
    if os.path.exists(img2_disk_path): os.remove(img2_disk_path)
    
    return mkpts0, mkpts1, mconf

## Kornia for Vis

In [ ]:
!pip install /kaggle/input/kornia-loftr/kornia-0.6.4-py2.py3-none-any.whl
!pip install /kaggle/input/kornia-loftr/kornia_moons-0.1.9-py3-none-any.whl

base_loftr_github = '/kaggle/input/loftrutils/LoFTR-master/LoFTR-master'
sys.path.append(base_loftr_github)
!pip install /kaggle/input/igla-py-wheels/loguru-0.5.3-py3-none-any.whl
!pip install /kaggle/input/igla-py-wheels/einops-0.4.1-py3-none-any.whl

## Loftr

In [ ]:
from src.loftr import LoFTR
from src.utils.plotting import make_matching_figures
from src.utils.comm import gather, all_gather
from src.utils.misc import lower_config, flattenList
from src.utils.profiler import PassThroughProfiler

from src.loftr import default_cfg
from src.config.default import get_cfg_defaults
import pytorch_lightning as pl

import kornia
from kornia_moons.feature import *
import kornia as K
import kornia.feature as KF


def draw_img_match(img1, img2, mkpts0, mkpts1, inliers):
    display_vertical = False#img1.shape[1]/img1.shape[0]>0.8 or img2.shape[1]/img2.shape[0]>0.8
    
    draw_LAF_matches(
    KF.laf_from_center_scale_ori(torch.from_numpy(mkpts0).view(1,-1, 2),
                                torch.ones(mkpts0.shape[0]).view(1,-1, 1, 1),
                                torch.ones(mkpts0.shape[0]).view(1,-1, 1)),

    KF.laf_from_center_scale_ori(torch.from_numpy(mkpts1).view(1,-1, 2),
                                torch.ones(mkpts1.shape[0]).view(1,-1, 1, 1),
                                torch.ones(mkpts1.shape[0]).view(1,-1, 1)),
    torch.arange(mkpts0.shape[0]).view(-1,1).repeat(1,2),
    K.tensor_to_image(img1),
    K.tensor_to_image(img2),
    inliers,
    draw_dict={'inlier_color': (0.2, 1, 0.2),
               'tentative_color': None, 
               'feature_color': (0.2, 0.5, 1), 'vertical': display_vertical})
    

def load_torch_kornia_image(fname, device, max_image_size):
    img,_ = load_resized_image(fname, max_image_size)
    img = K.image_to_tensor(img, False).float() /255.
    img = K.color.bgr_to_rgb(img)
    orig_img_device = img.to(device)
    return orig_img_device

## Utils

In [ ]:
def FlattenMatrix(M, num_digits=8):
    '''Convenience function to write CSV files.'''
    return ' '.join([f'{v:.{num_digits}e}' for v in M.flatten()])

In [ ]:
test_samples = []
with open(f'{src}/test.csv') as f:
    reader = csv.reader(f, delimiter=',')
    for i, row in enumerate(reader):
        # Skip header.
        if i == 0:
            continue
        test_samples += [row]

if dry_run:
    for sample in test_samples:
        print(sample)

In [ ]:
F_dict = {}
for i, row in enumerate(test_samples):
    sample_id, batch_id, image_1_id, image_2_id = row

    image_fpath_1 = f'{src}/test_images/{batch_id}/{image_1_id}.png'
    image_fpath_2 = f'{src}/test_images/{batch_id}/{image_2_id}.png'

    if dry_run:
        st = time.time()
        
    mkps1,mkps2,mconf = inf_qta(image_fpath_1, image_fpath_2, max_image_size=qta_max_img_size, divide_coef=32)
    
    if len(mkps1) > 7:
        rpt_F = 0.20
        conf_F = 0.999999
        m_iters_F = 200_000
        F, inliers_F = cv2.findFundamentalMat(mkps1, mkps2, cv2.USAC_MAGSAC, rpt_F, conf_F, m_iters_F)

        if dry_run:
            print("Running time: ", time.time() - st, " s")
            
        
        if F is None:
            F_dict[sample_id] = np.zeros((3, 3))
            continue
        else:
            F_dict[sample_id] = F
            inliers = inliers_F
    else:
        F_dict[sample_id] = np.zeros((3, 3))
        continue
            

    if dry_run and i < 3:
        F_inliers_total = 0 if F is None else (inliers_F == 1).sum()
        print('inliers F', F_inliers_total)
        
        orig_image_1 = load_torch_kornia_image(image_fpath_1, qta_torch_device, -1)
        orig_image_2 = load_torch_kornia_image(image_fpath_2, qta_torch_device, -1)
        draw_img_match(orig_image_1, orig_image_2, mkps1, mkps2, inliers)

    try:
        # cleanup
        gc.collect()
        torch.cuda.empty_cache()
        del mkps1, mkps2
    except Exception:
        pass

In [ ]:
%cd /kaggle/working

In [ ]:
if dry_run is False:
    !rm -rf  /kaggle/working/*

In [ ]:
with open('submission.csv', 'w') as f:
    f.write('sample_id,fundamental_matrix\n')
    for sample_id, F in F_dict.items():
        f.write(f'{sample_id},{FlattenMatrix(F)}\n')

if dry_run:
    !cat submission.csv